# Estimating Gentrification using Street View Images and Embeddings

This script (initially produced by ChatGPT) does the following (_this was my query_):
 - Read a spatial boundary file (that I will hard code)
 - Obtain the road network (from OSM?) for that area
 - Generate sample points on the road network roughly X meters apart
 - At each sample point, download the most recent street images for that location (either a single 360 degree view of a few smaller images). Use whichever API service is the most appropriate for obtaining the images. Importantly please record the date that the image was taken.
 - For each image, calculate an embedding using an appropriate foundation model (one that has been pre-trained to distinguish street environments specifically). Please use Hugging Face libraries.
 - _Do some analysis of the embeddings_ (Added by Nick later)
 - If necessary, calculate the mean embedding for each point (is this the best way to calculate a single embedding for a point represented by multiple images?)
 - Now, for each sampled point there will be a dataframe with information about the point and its embedding. Read another polygon spatial data file, that I will provide, which contains area-level estimates of gentrification.
 - Use point-in-polygon to get the gentrification (_or, more recently, deprivation) for each point.
 - Use cross-validation to train a couple of ML models (probaly random forest, linear regression and a neural network) to estimate gentrification from the embedding vectors
 - Choose the best model and parameter configuration and test this model on some held-out data.

## Configuration and library loading

In [ ]:
import os
import random
import base64
from pathlib import Path
import requests
import pickle
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import torch
from tqdm.auto import tqdm                    # auto picks notebook / console style
import multiprocessing
import json

from PIL import Image

# Spatial stuff
import folium
import shapely
from shapely.geometry import Point, LineString
from libpysal.weights import KNN
from esda.moran import Moran
import contextily as cx

# Machine learning imports
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans


# For caching models
from joblib import dump, load
from datetime import datetime


# Hugging Face Transformers for image embedding
from transformers import AutoImageProcessor, AutoModel  # will load a vision model

# Matplotlib for visualization
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex  # For creating hex colours


# ----------------- Configuration -----------------
DOWNLOAD_IMAGES = False  # If false then don't download any images, just load those that have been cached

# Can decide, after analysing images, which models we want to run (to predict gentrification and/or deprivation)
RUN_GENTRIFICATION_MODEL = False  
RUN_IMD_MODEL = False  # (This one is actually more about running a grid search to find the optimal model
                      # for the deprivation analysis. WHen False then it still runs models, but doesn't conduct the laborious search) 

# Data and cache dirs
data_dir = Path(os.path.join("..", "data", "airbnb-manchester"))
boundary_file = os.path.join(data_dir, "greater_manchester_lsoas.geojson")  # Path to boundary polygon file
gentrification_file = os.path.join("..", "data", "gmgi_data", "lsoa_summary_jan25.csv")  # Path to polygons with gentrification index
lsoas_file = os.path.join("..", "data", "LSOAs_2011", "LSOA_2011_EW_BSC_V4.shp")
imd_file = os.path.join("..", "data", "imd", "File_2_-_IoD2019_Domains_of_Deprivation.xlsx")

# For point sampling
density_per_km = 0.3  # number of points to sample per km of road
#density_per_km = 0.1  # VERY FEW WHILE TESTING
sample_spacing = 200.0   # distance in meters between sample points on roads
#sample_spacing = 5000.0  # Very large for testing
n_directions = 4         # number of street view images per point (e.g., 4 cardinal directions)
image_size = "640x640"   # requested image resolution from Street View API

# Create directories for caching if not exist
#Path(data_dir).mkdir(parents=True, exist_ok=True)
image_dir = Path(os.path.join(data_dir, "street_images"))
image_dir.mkdir(exist_ok=True)

## Data Loading

In [ ]:
# --- Load neighbourhood polygons and dissolve to one study-area boundary ---
boundary_neighs = gpd.read_file(boundary_file)
# (Optional) keep original neighbourhoods for later mapping/stratified sampling
#neighbourhoods_gdf = boundary_neighs.copy()
# Make sure we're in WGS84 (lat/lon) for OSM and APIs
boundary_neighs = boundary_neighs.to_crs(epsg=4326)
# Dissolve: merge all geometries into one polygon (MultiPolygon possible)
boundary_polygon = boundary_neighs.unary_union  # shapely (multi)polygon
boundary_gdf = gpd.GeoDataFrame(
    data={'name': ['study_area']},
    geometry=[boundary_polygon],
    crs=boundary_neighs.crs
)

# TEMP! Just keep one LSOA for testing
#boundary_gdf = boundary_neighs.sample(1)

print("Merged neighbourhoods into single study-area boundary.")
print("Bounds:", boundary_polygon.bounds)
boundary_gdf.plot(color='lightblue', edgecolor='black')


## Read the LSOA boundary data

(later it will be joined to the Greater Manchester Gentrification Index and IMD)

In [ ]:
lsoas =  gpd.read_file(lsoas_file)
manc_lads = ['Manchester', 'Rochdale', 'Bolton', 'Bury', 'Wigan', 'Oldham',  'Trafford', 'Salford', 'Tameside', 'Stockport']
manc_lads_pattern = '|'.join(manc_lads)
gm_lsoa=lsoas[lsoas['LSOA11NMW'].str.contains(manc_lads_pattern)]
gm_lsoa = gm_lsoa.to_crs(epsg=4326)
gm_lsoa.plot()

## Get Road Network for the Area from OSM

In [ ]:
# Get (or cache) the road network with OSMnx’s built-in GraphML I/O
# --------------------------------------------------------------------

# Tell OSMnx to keep all cache files inside the project folder (optional)
#ox.settings.cache_folder = str(data_dir / "osmnx_http_cache")  # HTTP/tile cache
#ox.settings.use_cache = True  # default is True
ox.settings.use_cache = False      # <- prevents the “cache” dir being written


graph_file = data_dir / "road_network.graphml"  # one self-contained file

if graph_file.exists():
    print("Loading road network from GraphML cache …")
    road_graph = ox.load_graphml(graph_file)
else:
    print("Downloading road network from OSM …")
    road_graph = ox.graph_from_polygon(boundary_polygon, network_type="drive")
    ox.save_graphml(road_graph, graph_file)
    print(f"Graph saved to {graph_file}")

# Convert to GeoDataFrame of edges for downstream sampling/plotting
edges_gdf = ox.graph_to_gdfs(road_graph, nodes=False)

print(f"Number of road segments: {len(edges_gdf)}")


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# Plot roads first (thin gray lines)
edges_gdf.plot(ax=ax, linewidth=0.4, color="gray")

# Plot the study-area outline on top (thicker red line)
boundary_gdf.boundary.plot(ax=ax, linewidth=2, edgecolor="red")

ax.set_title("Road network & study-area boundary", pad=12)
ax.set_axis_off()          # hides lat/lon ticks for a cleaner look
ax.set_aspect("equal")     # keeps the map from looking stretched

plt.show()

## Generate a Sample of Points along the road network

In [ ]:
if DOWNLOAD_IMAGES:
    # Generate a sample of points along the street network (probabilistic)
    # --------------------------------------------------------------------
    # PARAMETERS
    min_points_per_edge = 0  # allow very short edges to have none
    max_points_per_edge = 10  # safety cap per edge

    # Re-project roads to a metric CRS for length calculations
    utm_crs = boundary_gdf.estimate_utm_crs()
    edges_proj = edges_gdf.to_crs(utm_crs)

    samples = []

    # Loop over every edge geometry
    for geom in edges_proj.geometry:
        if geom is None or geom.length == 0:
            continue

        length_m = geom.length
        expected = length_m / 1000 * density_per_km  # λ for Poisson
        n_points = np.random.poisson(expected)  # 0, 1, 2, …

        # Clip to limits
        n_points = max(min_points_per_edge, min(n_points, max_points_per_edge))

        if n_points == 0:
            continue

        # Evenly distribute interior points (skip endpoints)
        distances = np.linspace(0, length_m, n_points + 2)[1:-1]
        for d in distances:
            samples.append(geom.interpolate(d))

    # Build GeoDataFrame of sample points (back to WGS-84 for API use)
    points_gdf = (
        gpd.GeoDataFrame(geometry=gpd.GeoSeries(samples), crs=utm_crs)
        .to_crs(epsg=4326)
    )
    points_gdf["lon"] = points_gdf.geometry.x.round(6)
    points_gdf["lat"] = points_gdf.geometry.y.round(6)
    points_gdf = points_gdf.drop_duplicates(subset=["lat", "lon"]).reset_index(drop=True)

    print(
        f"Generated {len(points_gdf)} points "
        f"with expected density {density_per_km} pts/km."
    )

else:
    print("DOWNLOAD_IMAGES is false, so not sampling from the road network")

In [ ]:
if DOWNLOAD_IMAGES:
    fig, ax = plt.subplots(figsize=(8, 8))

    # Plot roads first (thin gray lines)
    edges_gdf.plot(ax=ax, linewidth=0.4, color="gray")

    # Plot the sample points
    points_gdf.plot(ax=ax, color="blue", markersize=2, label="Sample Points")

    # Plot the study-area outline on top (thicker red line)
    boundary_gdf.boundary.plot(ax=ax, linewidth=2, edgecolor="red")

    ax.set_title("Road network & study-area boundary", pad=12)
    ax.set_axis_off()          # hides lat/lon ticks for a cleaner look
    ax.set_aspect("equal")     # keeps the map from looking stretched

    plt.show()

## Download street view images for each point

Note: expects a valid Google Maps API key in the file `google_maps_api_key.txt` in the same directory as this script (not synced to github for obvious reasons).

In [ ]:
# Get the API key from a file (of needed)
if DOWNLOAD_IMAGES:
    with open('google_maps_api_key.txt', 'r') as f:
        api_key = f.readline().strip()

A load of images were downloaded incorrectly (just got a streetview blank jpeg not a proper image). The following code identifies the first image that went wrong (I found that it was image 8116 by looking at the saved picture files) and removes it and all other from the point records. Then I delete the corresponding images. Hopefully this removed the bad images and kept the point cache and images aligned.

In [ ]:
#with open(points_data_cache, "rb") as f:
#    point_records = pickle.load(f)
#
#point_records_bak = point_records.copy()
#index = next(i for i, d in enumerate(point_records) if d['point_id'] == 19196)
#print(index)
#point_records[index]  # This is the index of the record identified above
#
#point_records = point_records[:index]
#
#point_records[18895]
#
#with open(points_data_cache, "wb") as f:
#    pickle.dump(point_records, f)


In [ ]:
# Cache file for the entire points data with embeddings (images are stored separately)
DEBUG = False
points_data_cache = data_dir / "points_with_embeddings.pkl"

# -----------------------------------------------------------
# Load existing cache so we can *append* new sample points
# -----------------------------------------------------------
if points_data_cache.exists():
    print("Loading cached point data …")
    with open(points_data_cache, "rb") as f:
        point_records = pickle.load(f)
    existing_coords = {(rec["latitude"], rec["longitude"]) for rec in point_records}
    next_id = max(rec["point_id"] for rec in point_records) + 1
else:
    point_records = []
    existing_coords = set()
    next_id = 0

print(f"Cache currently has {len(point_records)} points.")
added_this_run = 0

# -----------------------------------------------------------
# Iterate through newly‑sampled street‑network points (with a progress bar)
# -----------------------------------------------------------
if DOWNLOAD_IMAGES:
    for _, row in tqdm(points_gdf.iterrows(), total=len(points_gdf), desc="Downloading images"):
        lat = row["lat"]
        lon = row["lon"]

        # Skip if imagery for this coordinate (rounded earlier) already exists
        if (lat, lon) in existing_coords:
            continue

        point_id = next_id
        next_id += 1
        added_this_run += 1

        # ---- Street‑View metadata ----
        meta_params = {"location": f"{lat},{lon}", "key": api_key}
        meta_url = "https://maps.googleapis.com/maps/api/streetview/metadata"
        try:
            meta = requests.get(meta_url, params=meta_params).json()
        except Exception as e:
            print(f"[Point {point_id}] Metadata request failed: {e}")
            continue

        if meta.get("status") != "OK":
            if DEBUG:
                print(f"[Point {point_id}] Street View not available (status={meta.get('status')}).")
            continue

        pano_id = meta.get("pano_id")
        date = meta.get("date")  # e.g. "2024‑08"

        # ---- Download images for the specified headings ----
        point_image_files = []
        for heading in np.linspace(0, 360, num=n_directions, endpoint=False):
            fname = f"point{point_id}_heading{int(heading)}.jpg"
            image_path = image_dir / fname
            point_image_files.append(str(image_path))

            if image_path.exists():
                continue  # already on disk

            img_params = {
                "size": image_size,
                "pano": pano_id,
                "heading": str(int(heading)),
                "pitch": "0",
                "key": api_key,
            }
            img_url = "https://maps.googleapis.com/maps/api/streetview"
            try:
                img_resp = requests.get(img_url, params=img_params)
                with open(image_path, "wb") as f:
                    f.write(img_resp.content)
            except Exception as e:
                print(f"Failed to download image for point {point_id}, heading {heading}: {e}")

        # ---- Append the new record ----
        point_records.append(
            {
                "point_id": point_id,
                "latitude": lat,
                "longitude": lon,
                "date": date,
                "image_files": point_image_files,
                "embedding": None,  # to be filled later
            }
        )
        existing_coords.add((lat, lon))

    print(f"Added {added_this_run} new points this run (total now {len(point_records)}).")

    # Persist the (possibly) updated cache immediately
    with open(points_data_cache, "wb") as f:
        pickle.dump(point_records, f)

else:
    print("DOWNLOAD_IMAGES set to false, so not downloading any images")

Map of the full sample (cache + any others just downlaoded)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# Plot roads first (thin gray lines)
#edges_gdf.to_crs(epsg=3857).plot(ax=ax, linewidth=0.2, color="grey")

# Plot the sample points (note the on-the-fly projection to Web Mercator for Contextily later)
gpd.GeoDataFrame(point_records,
                 geometry=[Point(rec["longitude"], rec["latitude"]) for rec in point_records],
                 crs="EPSG:4326"
                 ).to_crs(epsg=3857).plot(ax=ax, color="blue", markersize=3, label="Sample Points")

# Plot the study-area outline on top (thicker red line)
boundary_gdf.to_crs(epsg=3857).boundary.plot(ax=ax, linewidth=2, edgecolor="red")

# Basemap
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

ax.set_title("Greater Manchester road network and street view sample locations", pad=12)
ax.set_axis_off()          # hides lat/lon ticks for a cleaner look
ax.set_aspect("equal")     # keeps the map from looking stretched


# Save the figure as PNG
plt.savefig("img/gm_street_images.png", dpi=300, bbox_inches="tight")  # High resolution and tight layout


plt.show()

Show some randomly chosen images

In [ ]:
# -------------------------------------------------------
# CONFIG
# -------------------------------------------------------
n_points_to_show = 6          # rows
headings_per_pt  = n_directions   # columns; assumption: same for all points

# -------------------------------------------------------
# SAMPLE POINTS
# -------------------------------------------------------
# point_records was created earlier when you downloaded images
valid_records = [rec for rec in point_records if rec.get("image_files")]
if len(valid_records) < n_points_to_show:
    raise ValueError(f"Need at least {n_points_to_show} points with images; "
                     f"found {len(valid_records)}")

sample_pts = random.sample(valid_records, n_points_to_show)

# -------------------------------------------------------
# PLOT
# -------------------------------------------------------
fig, axes = plt.subplots(n_points_to_show,
                         headings_per_pt,
                         figsize=(headings_per_pt * 3.5, n_points_to_show * 3))

for row, rec in enumerate(sample_pts):
    imgs = rec["image_files"]
    # If fewer than expected headings (e.g. download failure), pad with blanks
    while len(imgs) < headings_per_pt:
        imgs.append(None)

    for col, img_path in enumerate(imgs[:headings_per_pt]):
        ax = axes[row, col] if n_points_to_show > 1 else axes[col]

        if img_path and os.path.exists(img_path):
            ax.imshow(Image.open(img_path))
        else:
            # blank panel if the image is missing
            ax.text(0.5, 0.5, "no image", ha="center", va="center")
        ax.axis("off")

        # column headers once at top
        if row == 0:
            ax.set_title(f"heading {col*360/headings_per_pt:.0f}°", fontsize=10, pad=6)

    # label the leftmost image with point info
    axes[row, 0].set_ylabel(
        f"point {rec['point_id']}\n({rec['latitude']:.3f}, {rec['longitude']:.3f})",
        fontsize=8, rotation=0, ha="right", va="center"
    )

plt.suptitle("Street-View snapshots: 6 random points × 4 headings", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()


## Compute the Embeddings

(Note: would like to use Places365 but not available in Hugging Face yet, so using ViT base model instead)

In [ ]:
# -------------------------------------------------------------
# Load ViT-Base (ImageNet-21k) and pick CUDA ▸ MPS ▸ CPU device
# -------------------------------------------------------------

model_name = "google/vit-base-patch16-224-in21k"
print(f"Loading {model_name} …")
processor   = AutoImageProcessor.from_pretrained(model_name)
model       = AutoModel.from_pretrained(model_name).eval()   # no classifier head

# ----- smart device selection -----
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("✔ Using CUDA GPU")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")      # Apple-Silicon Metal
    print("✔ Using Apple MPS GPU")
else:
    device = torch.device("cpu")
    print("✔ Using CPU")

model.to(device)

# -------------------------------------------------------------
# Embed all images for each point
# -------------------------------------------------------------
already_have_embedding = 0
for rec in point_records:
    embeds = []
    # Skip records that already have an embedding
    if rec.get("embedding") is not None:
        already_have_embedding += 1
        continue
    embeds = []
    for img_path in rec["image_files"]:
        if not os.path.exists(img_path):
            continue
        # Load & convert to RGB (some JPEGs are encoded as P)
        img = Image.open(img_path).convert("RGB")

        # Tokenize and normalize the image
        inputs = processor(img, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}   # move tensors

        # Forward pass through the model (no gradients needed)
        with torch.no_grad():
            out = model(**inputs)

        # Extract the CLS token embedding (first token in the sequence)
        cls = out.last_hidden_state[0, 0, :].cpu().numpy()     # CLS token
        embeds.append(cls)

    # Store the mean embedding for this point (or none)
    rec["embedding"] = None if not embeds else np.mean(embeds, axis=0)

print(f"Created {len(point_records)-already_have_embedding} new embeddings. "
      f"{already_have_embedding} points had existing embeddings.")

# cache the enriched records
with open(points_data_cache, "wb") as f:
    pickle.dump(point_records, f)

print(f"✓ {len(point_records)} image embeddings computed and cached.")


## Analyse the embeddings

No idea what these show really, so do some anaysis to see how they vary over space etc.

### PCA embeddings map

Do a PCA to get 3 dimensions for each embedding, the scale to RGB and map them. Click on a point to see the images.

In [ ]:
# Visualise embeddings in “RGB PCA space”
# Each point’s 768-D embedding → 3-D PCA → scaled 0-1 → RGB colour
# --------------------------------------------------------------------


# -------------------------------------------------------
# 1. Collect embeddings and GeoDataFrame of points
# -------------------------------------------------------
records = [rec for rec in point_records if rec["embedding"] is not None]
X = np.vstack([rec["embedding"] for rec in records])  # (N, 768)
coords = np.array([[rec["longitude"], rec["latitude"]] for rec in records])

# optional: bring in points_gdf if you prefer plotting via GeoPandas
# points_gdf = gpd.GeoDataFrame({'geometry': gpd.points_from_xy(coords[:,0], coords[:,1])})

# -------------------------------------------------------
# 2. PCA → first 3 components
# -------------------------------------------------------
pca = PCA(n_components=3, random_state=0)
rgb3 = pca.fit_transform(X)  # (N, 3)

# -------------------------------------------------------
# 3. Scale each PC separately to [0, 1]
# -------------------------------------------------------
rgb_min = rgb3.min(axis=0)  # per-column min
rgb_max = rgb3.max(axis=0)
rgb_scaled = (rgb3 - rgb_min) / (rgb_max - rgb_min + 1e-9)  # avoid /0
colors = [tuple(c) for c in rgb_scaled]  # list of (r,g,b) floats 0-1
hex_colors = [to_hex(tuple(c)) for c in rgb_scaled]             # '#rrggbb'

# -------------------------------------------------------
# 4. Map to plot on lon/lat, coloured by PCA-RGB
# -------------------------------------------------------


# ---- 4) Build GeoDataFrame (WGS84) ----
gdf = gpd.GeoDataFrame(
    data = { "pc1": rgb_scaled[:, 0], "pc2": rgb_scaled[:, 1], "pc3": rgb_scaled[:, 2], "hex": hex_colors, },
    geometry=gpd.points_from_xy(coords[:, 0], coords[:, 1]),
    crs="EPSG:4326",
)

# ---- 5) Base map via gpd.explore, then add per-point colours with Folium ----
# (explore() returns a Folium map; we start with an empty layer for nice tiles/controls)
m = gdf.iloc[:0].explore(tiles="CartoDB positron", control_scale=True)

# add one CircleMarker per point using the precomputed PCA colour
for _, r in gdf.iterrows():
    folium.CircleMarker(
        location=(r.geometry.y, r.geometry.x),
        radius=4,
        color=r["hex"],
        fill=True,
        fill_color=r["hex"],
        fill_opacity=1.0,
        weight=0,
        tooltip=f"PCs: ({r.pc1:.2f}, {r.pc2:.2f}, {r.pc3:.2f})",
    ).add_to(m)

minx, miny, maxx, maxy = gdf.total_bounds
m.fit_bounds([[miny, minx], [maxy, maxx]])

m  # display in notebook / JupyterLab

# Alternative: matplotlib static plot
#fig, ax = plt.subplots(figsize=(7, 7))
#ax.scatter(coords[:, 0], coords[:, 1], c=colors, s=8, alpha=0.9, linewidths=0)
#
#ax.set_aspect("equal")
#ax.set_title("Point embeddings visualised as RGB (PCA first 3 comps)")
#ax.axis("off")  # hide ticks – remove if you want lon/lat grid
#
#plt.show()



Map a small sample and display the images too

In [ ]:
#MAP_SAMPLE_SIZE = 500  # Takes a while and usually crashes jupyter
MAP_SAMPLE_SIZE = 200  # Easy to plot

def img_tag(path, width=150):
    """
    Return a <img> tag with the image inlined as base64.
    Width in pixels; height auto.
    """
    if not os.path.exists(path):
        return "<div style='width:{}px;height:{}px;background:#ccc;'>no img</div>".format(width, int(width*0.75))
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"<img src='data:image/jpeg;base64,{b64}' width='{width}px' style='margin:2px;'/>"


m = folium.Map(
    location=[coords[:, 1].mean(), coords[:, 0].mean()], # centred on the mean lat/lon
    zoom_start=12,
    tiles="cartodbpositron"   # light background; try 'openstreetmap'
)

# ---------- Add points ----------
#for rec, hx in zip(records, hex_colors):
# Sample ome points, map breaks with too many
for rec, hx in random.sample(list(zip(records, hex_colors)), MAP_SAMPLE_SIZE):
    # Build HTML table of up to 4 images (N,E,S,W)
    imgs_html = "".join(img_tag(p, width=140) for p in rec["image_files"][:4])
    popup_html = f"""
    <div style="text-align:center">
        <b>point {rec['point_id']}</b><br/>
        {imgs_html}
    </div>
    """
    folium.CircleMarker(
        location=[rec["latitude"], rec["longitude"]],
        radius=6,
        color=hx, fill=True, fill_color=hx, fill_opacity=0.9, weight=0,
        popup=folium.Popup(popup_html, max_width=600)
    ).add_to(m)

folium.LayerControl().add_to(m)
#m.save("embedding_rgb_with_photos.html")
m

Look at how similar an embedding is to it's K nearest neighbours

### Embedding similarity maps

Look at how similar the point embeddings are to each other

In [ ]:
# Find the nearest neighbours for each point
nbrs = NearestNeighbors(n_neighbors=6).fit(coords)
_, idxs = nbrs.kneighbors(coords)

# Calculate the mean cosine similarity to the nearest neighbours (excluding self)
gdf['local_sim'] = [np.mean(cosine_similarity([X[i]], X[idxs[i,1:]])[0]) for i in range(len(X))]

In [ ]:
# Map the point similarity (not especially helpful)
gdf.explore(column='local_sim', cmap='viridis', fit_bounds=True)

In [ ]:
# Aggreate similarity to a regular grid; more useful

# project to a metric CRS first (so grid cell sizes are meaningful)
gdf = gdf.to_crs(3857)  # https://epsg.io/3857

# --- 1. Make grid ---
cell_size = 2000    # metres
xmin, ymin, xmax, ymax = gdf.total_bounds  # (need to do again with new crs)

cols = np.arange(xmin, xmax + cell_size, cell_size)
rows = np.arange(ymin, ymax + cell_size, cell_size)

polys = [
    shapely.geometry.box(x, y, x + cell_size, y + cell_size)
    for x in cols for y in rows
]
grid = gpd.GeoDataFrame({'geometry': polys}, crs=gdf.crs)

# --- 2. Spatial join + aggregate ---
joined = gpd.sjoin(gdf, grid, predicate="within")

#agg = joined.groupby('index_right')['local_sim'].mean().reset_index()
agg = (joined.groupby('index_right', observed=True)['local_sim'].mean())

#grid['mean_sim'] = agg['local_sim']
grid['mean_sim'] = np.nan
grid.loc[agg.index, 'mean_sim'] = agg.values


In [ ]:
ax = grid.plot(column='mean_sim', cmap='viridis', legend=True)
#gdf.plot(ax=ax)

In [ ]:
# --- 3. Plot interactively --- (turned off for now)

# The grid
#m = grid.explore(column='mean_sim', cmap='viridis', legend=True, fit_bounds=True)

# Add the points
#gdf.explore(m=m, column='local_sim', cmap='viridis', marker_kwds=dict(radius=2, opacity=0.4, fill=True))

#m

Not particularly conclusive

### Embedding similarity over distance (_not sure what this does!_).

_I'd like to see how the embeddings change with distance. My hypothesis is that for any given point, it will be quite similar to its neighbours, but will also be very similar to some distant points as well. I thought about using Moran's I, but it is univariate, so can't be calculated on dense embeddings. ChatGPT suggested a Mantel Correlogram which compares Euclidean with cosine similarity over distance bands, but I'm not sure what that does either really. I does make a graph though! I'm leaving the code in below but am going to move on to look at clustering embeddings._

_Similarly with the following ChatGPT code, it calculates similarity over distance bands and then plots these distances and the uncertainty (mean and IQR). May be useful later but again I'm not confident in what it does so am not using it at this stage_

### Clustering Embeddings

If we cluster embeddings then there is some interesting analysis that can follow.

Start by deciding on the appropriate number of clusters (thanks ChatGPT).

In [ ]:


# -----------------------------------------------------------
# X : (N, D) embedding matrix (e.g., from your Street View model)
# We'll try a range of candidate cluster counts.
# -----------------------------------------------------------
K_range = range(2, 16)  # test 2→15 clusters

inertia = []   # total within-cluster sum of squares (elbow method)
silh = []      # silhouette (higher = better separation)
ch = []        # Calinski–Harabasz (higher = better)
db = []        # Davies–Bouldin (lower = better)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(X)
    labels = km.labels_

    inertia.append(km.inertia_)
    silh.append(silhouette_score(X, labels))
    ch.append(calinski_harabasz_score(X, labels))
    db.append(davies_bouldin_score(X, labels))

# Collect results for inspection
df_scores = pd.DataFrame({
    "k": K_range,
    "inertia": inertia,
    "silhouette": silh,
    "calinski_harabasz": ch,
    "davies_bouldin": db
})

# -----------------------------------------------------------
# Plot diagnostic curves
# -----------------------------------------------------------
fig, ax1 = plt.subplots()
ax1.plot(df_scores["k"], df_scores["inertia"], "-o", color="tab:gray", label="Inertia (↓)")
ax1.set_xlabel("Number of clusters (k)")
ax1.set_ylabel("Inertia (total within-cluster variance)", color="tab:gray")
ax1.tick_params(axis="y", labelcolor="tab:gray")

ax2 = ax1.twinx()
ax2.plot(df_scores["k"], df_scores["silhouette"], "-o", color="tab:blue", label="Silhouette (↑)")
ax2.plot(df_scores["k"], df_scores["calinski_harabasz"]/np.max(df_scores["calinski_harabasz"]),
         "--o", color="tab:green", label="Calinski–Harabasz (↑, scaled)")
ax2.plot(df_scores["k"], 1/np.array(df_scores["davies_bouldin"]),
         "-.o", color="tab:red", label="1 / Davies–Bouldin (↑)")
ax2.set_ylabel("Relative cluster quality (higher = better)")
ax2.grid(alpha=0.3)

# combine legends
lines, labels = [], []
for a in (ax1, ax2):
    L, lab = a.get_legend_handles_labels()
    lines += L; labels += lab
ax1.legend(lines, labels, loc="best")

plt.title("Cluster-number diagnostics for embedding space")
plt.tight_layout()
plt.show()

# -----------------------------------------------------------
# Interpret
# -----------------------------------------------------------
# - "Elbow" point on the inertia curve = plausible k (diminishing returns).
# - Silhouette / CH peak and DB minimum should roughly coincide.
# Choose k where those metrics stabilise or peak.


It isn't conclusive, so go with _N=8_ for now

In [ ]:
n_clusters = 8

Calculate the clusters

In [ ]:
kmeans = KMeans(n_clusters=n_clusters).fit(X)
gdf['cluster'] = kmeans.labels_.astype(str)

gdf

Map the clusters

In [ ]:
m = gdf.explore(
    column="cluster",
    categorical=True,
    categories=sorted(gdf["cluster"].unique()),
    cmap="tab20",                 # good categorical palette
    legend=True,
    legend_kwds={"caption": "Embedding clusters"},
    marker_kwds={"radius": 3, "fillOpacity": 1.0},
    style_kwds={"weight": 0},
    fit_bounds=True,
    tiles = "CartoDB positron"
)
m 

Estimate degree to which classes are clustered, controlling for the arbitrary spatial arrangement of the points. Conclusion: looks like there is clustering **but need to check the code before doing anything with this**.

In [ ]:
# ------------------------------------------------------------
# 1) Join-count test on a k-NN graph (random-labelling null)
#    - Builds an undirected k-NN graph from coordinates
#    - Counts same-class edges overall and per class
#    - Uses permutations to get p-values
# ------------------------------------------------------------
from collections import Counter, defaultdict

# Inputs:
#   gdf["cluster"] : integer labels from your embedding clustering
#   gdf.geometry   : points (use projected CRS; metres not required here but OK)
labels = gdf["cluster"].to_numpy()
coords = np.c_[gdf.geometry.x, gdf.geometry.y]
N = len(gdf)

k = n_clusters        # neighbours per node (tune)
B = 999               # permutations for p-values (e.g., 999)

# --- build undirected k-NN edge list (as pairs of indices) ---
nbrs = NearestNeighbors(n_neighbors=k+1).fit(coords)  # +1 includes self
_, idxs = nbrs.kneighbors(coords)
edges = set()
for i in range(N):
    for j in idxs[i, 1:]:    # skip self at position 0
        a, b = (i, j) if i < j else (j, i)
        edges.add((a, b))    # undirected: store once
edges = np.array(list(edges), dtype=int)
E = len(edges)

# --- observed join counts ---
ell = labels
same = (ell[edges[:,0]] == ell[edges[:,1]])
JC_all_obs = int(np.sum(same))  # total same-class joins

# per-class same-class joins
classes = np.unique(ell)
JC_cls_obs = {c: int(np.sum((ell[edges[:,0]] == c) & (ell[edges[:,1]] == c)))
              for c in classes}

# --- permutation test (random labelling) ---
rng = np.random.default_rng(0)
JC_all_null = np.zeros(B, dtype=int)
JC_cls_null = {c: np.zeros(B, dtype=int) for c in classes}

for b in range(B):
    perm = rng.permutation(N)
    lp = ell[perm]
    same_p = (lp[edges[:,0]] == lp[edges[:,1]])
    JC_all_null[b] = int(np.sum(same_p))
    for c in classes:
        JC_cls_null[c][b] = int(np.sum((lp[edges[:,0]] == c) & (lp[edges[:,1]] == c)))

# --- p-values (one-sided: clustered = more same-class joins than null) ---
def perm_pval(obs, null):
    return (np.sum(null >= obs) + 1) / (len(null) + 1)

p_all = perm_pval(JC_all_obs, JC_all_null)
p_cls = {c: perm_pval(JC_cls_obs[c], JC_cls_null[c]) for c in classes}

# --- tidy summary ---
out_rows = [{"stat": "ALL", "obs_same_joins": JC_all_obs,
             "exp_mean": float(JC_all_null.mean()),
             "p_value": float(p_all)}]
for c in classes:
    out_rows.append({"stat": f"class_{int(c)}",
                     "obs_same_joins": JC_cls_obs[c],
                     "exp_mean": float(JC_cls_null[c].mean()),
                     "p_value": float(p_cls[c])})
joincount_results = pd.DataFrame(out_rows)
print(joincount_results)


In [ ]:
WHAT NOW?!?

## Prepare training data for the gentrification model

Note that we don't actually calculate the model later unless `RUN_GENTRIFICATION_MODEL` is true, mostly because it takes bloody ages. But some of the operations, like joining LSOAs to embeddings, are needed for the IMD model so we do run some of this. 

In [ ]:
# Load GMGI
gmgi = pd.read_csv(gentrification_file)
gmgi = gmgi.iloc[:,1:]  # Drop the first column
gmgi

Attach GMGI to LSOAs that we read earlier

In [ ]:
# Attach GMGI columns to lsoa data
gm_gmgi_lsoa = pd.merge(left=gm_lsoa, right=gmgi, left_on="LSOA11CD", right_on="LSOA11CD")
# Map the gentrification index (sanity check)
gm_gmgi_lsoa.plot(column="gi_n")

Join image embeddings points to gentrification LSOAs (so we have the 'ground truth' gentrification score).

In [ ]:
# Check all records have an embeding
invalid_records = [rec for rec in point_records if rec.get('embedding') is None]
assert len(invalid_records)==0, f"Found {len(invalid_records)} invalid points"

# Now join the embeddings points to the LSOAs
point_coords = [Point(rec['longitude'], rec['latitude']) for rec in point_records]
points_labels_gdf = gpd.GeoDataFrame(point_records, geometry=point_coords, crs="EPSG:4326")

# Perform spatial join to get gentrification label for each point
points_labels_gdf = gpd.sjoin(points_labels_gdf, gm_gmgi_lsoa, how='inner', predicate='within')
# sjoin may add an index from the polygon ('index_right'); we can drop it
if 'index_right' in points_labels_gdf.columns:
    points_labels_gdf = points_labels_gdf.drop(columns=['index_right'])

print(f"Points after spatial join: {len(points_labels_gdf)} / {len(point_records)}"
      f" `(some points may lie outside the label polygons and were dropped)")

Show them on a map

In [ ]:
label_col = 'gi_n'

In [ ]:
ax = gm_gmgi_lsoa.plot(column=label_col)
# Add the points
points_labels_gdf.plot(ax=ax, column=label_col, markersize=10, edgecolor="black", linewidth=0.5)

Prepare X and y data for the ML model

In [ ]:
# ------------------------------------------------------------------
# Validate point records before building X, y
# ------------------------------------------------------------------
def _is_bad_embedding(e):
    """True if embedding is missing, not a numpy array, or empty."""
    return (e is None) or (not isinstance(e, np.ndarray)) or (e.size == 0)


# Boolean masks
missing_label  = points_labels_gdf[label_col].isna()
bad_embedding  = points_labels_gdf["embedding"].apply(_is_bad_embedding)

# Report any problems
n_bad_label   = missing_label.sum()
n_bad_embed   = bad_embedding.sum()
n_bad_total   = (missing_label | bad_embedding).sum()

if n_bad_total:
    msg = (f"⚠️  {n_bad_total} invalid point(s) detected "
           f"({n_bad_label} with missing label, "
           f"{n_bad_embed} with missing/empty embedding).")
    print(msg)

    # show first few offending rows for inspection
    print(points_labels_gdf.loc[missing_label | bad_embedding,
                                ["point_id", label_col, "embedding"]].head())

else:
    print(f"{n_bad_total} invalid point(s) detected ")

# ------------------------------------------------------------------
# Keep only valid rows
# ------------------------------------------------------------------
points_labels_gdf = points_labels_gdf.loc[~(missing_label | bad_embedding)].reset_index(drop=True)

if points_labels_gdf.empty:
    raise ValueError("No valid points left after cleaning — cannot train model.")

# ------------------------------------------------------------------
# Build feature matrix X and target vector y
# ------------------------------------------------------------------
X = np.stack(points_labels_gdf["embedding"].values)     # shape (n_points, embed_dim)
y = points_labels_gdf[label_col].values

print("Feature matrix shape:", X.shape, "Target vector shape:", y.shape)


## Run the gentrification ML models

To predict gentrification from the image embeddings

In [ ]:
if RUN_GENTRIFICATION_MODEL:
    # Split data into training and test sets
    X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
        X, y, np.arange(X.shape[0]), test_size=0.2, random_state=42)
    print(f"Training points: {X_train.shape[0]}, Test points: {X_test.shape[0]}")
    
    # Define model pipelines and parameter grids for cross-validation
    models = []
    param_grids = []
    
    # 1. Linear Regression (with standard scaling)
    pipe_linear = Pipeline([
        ('scaler', StandardScaler()),
        ('reg', LinearRegression())
    ])
    # No hyperparameters to tune for plain LinearRegression (we could consider Ridge/Lasso alphas, but skip for simplicity)
    models.append(pipe_linear)
    param_grids.append({})  # empty grid means just evaluate the baseline linear model
    
    # 2. Random Forest Regressor
    pipe_rf = Pipeline([
        ('scaler', StandardScaler()),  # scaler doesn't affect RF but included for uniformity
        ('reg', RandomForestRegressor(random_state=42))
    ])
    models.append(pipe_rf)
    param_grids.append({
        'reg__n_estimators': [100, 200],   # try 100 and 200 trees
        'reg__max_depth': [None, 10, 20]  # try unlimited depth and a couple of depth limits
    })
    
    # 3. Neural Network (MLPRegressor)
    pipe_mlp = Pipeline([
        ('scaler', StandardScaler()),
        ('reg', MLPRegressor(max_iter=500, random_state=42))
    ])
    models.append(pipe_mlp)
    param_grids.append({
        'reg__hidden_layer_sizes': [(100,), (100,50)],  # one hidden layer vs two layers
        'reg__alpha': [1e-4, 1e-3]  # L2 regularization strengths
        # (Other hyperparameters like learning_rate_init can be added if needed)
    })
    
    # Perform cross-validation for each model to find the best hyperparameters
    best_model = None
    best_score = -np.inf
    best_model_name = None
    
    print("Training models")
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    ncores = min(multiprocessing.cpu_count()-1, 100)
    print(f"USing {ncores} cores.")  # Take all cores but one, and not more than 100
    for model, param_grid, name in zip(models, param_grids, ["LinearReg", "RandomForest", "NeuralNet"]):
        print(f"\tTraining: {model}...")
        if param_grid:
            # Use GridSearchCV for models with hyperparameters
            grid = GridSearchCV(model, param_grid, cv=cv, scoring='r2', n_jobs=ncores)
            grid.fit(X_train, y_train)
            cv_score = grid.best_score_
            model_best = grid.best_estimator_
            params_best = grid.best_params_
        else:
            # For Linear Regression (no params to tune), just do cross_val_score
            scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
            cv_score = np.mean(scores)
            model.fit(X_train, y_train)  # train on full training data
            model_best = model
            params_best = None
        print(f"{name} CV mean R^2 = {cv_score:.3f} {('(best params: '+str(params_best)+')') if params_best else ''}")
        # Track the best model
        if cv_score > best_score:
            best_score = cv_score
            best_model = model_best
            best_model_name = name
    
    print(f"Best model from CV: {best_model_name} with R^2 = {best_score:.3f}")
    
else:
    print("RUN_GENTRIFICATION_MODEL is False so not running the gentrification predictor model")

Test on a held out set:

In [ ]:
if RUN_GENTRIFICATION_MODEL:
    # Ensure best_model is trained on the entire training set (GridSearchCV already did refit; for linear we did manually)
    # If best_model was from cross_val_score (Linear), we already called model.fit above.
    
    # Predict on test set
    y_pred = best_model.predict(X_test)
    
    # Evaluate performance
    r2_test = r2_score(y_test, y_pred)
    # rmse_test = mean_squared_error(y_test, y_pred, squared=False)  # (not avialable in older skikitlearn)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"\nTest R^2 score: {r2_test:.3f}")
    print(f"Test RMSE: {rmse_test:.3f}")
    
    # Attach predictions to the test points for mapping
    test_points = points_labels_gdf.iloc[test_idx].copy()
    test_points['predicted'] = y_pred
    test_points['error'] = test_points['predicted'] - test_points[label_col]

Visualise gentrification maps and error predictions

In [ ]:
if RUN_GENTRIFICATION_MODEL:
    # Plot the actual gentrification by area
    fig, ax = plt.subplots(figsize=(8, 6))
    gm_gmgi_lsoa.plot(column=label_col, ax=ax, legend=True, cmap='plasma', edgecolor='gray')
    ax.set_title("Actual Gentrification by Area")
    ax.axis('off')
    plt.show()
    
    # Plot the prediction errors at test points
    fig, ax = plt.subplots(figsize=(8, 6))
    # Plot polygons outlines for context
    gm_gmgi_lsoa.boundary.plot(ax=ax, color='lightgray')
    # Plot test points with error coloration
    test_points.plot(column='error', ax=ax, legend=True, cmap='coolwarm', markersize=50)
    ax.set_title("Model Prediction Error at Test Points")
    ax.axis('off')
    plt.show()

## Prepare data for the deprivation model

Repeat the gentrification stuff but this time try to predict deprivation (measured using the IMD).

Get the IMD data and read it in. I am using the file [File_2_-_IoD2019_Domains_of_Deprivation.xlsx](../data/File_2_-_IoD2019_Domains_of_Deprivation.xlsx) from the main [IMD 2019 gov page](https://www.gov.uk/government/statistics/english-indices-of-deprivation-2019)

In [ ]:

# Read Excel file
imd = pd.read_excel(imd_file, sheet_name="IoD2019 Domains", header=0)

# Rename columns to simpler versions
imd_col_map = {
    "LSOA code (2011)": "lsoa_2011_code",
    "LSOA name (2011)": "lsoa_2011_name",
    "Local Authority District code (2019)": "lad_2019_code",
    "Local Authority District name (2019)": "lad_2019_name",
    "Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)": "imd_rank",
    "Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)": "imd_decile",
    "Income Rank (where 1 is most deprived)": "income_rank",
    "Income Decile (where 1 is most deprived 10% of LSOAs)": "income_decile",
    "Employment Rank (where 1 is most deprived)": "employment_rank",
    "Employment Decile (where 1 is most deprived 10% of LSOAs)": "employment_decile",
    "Education, Skills and Training Rank (where 1 is most deprived)": "education_rank",
    "Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)": "education_decile",
    "Health Deprivation and Disability Rank (where 1 is most deprived)": "health_rank",
    "Health Deprivation and Disability Decile (where 1 is most deprived 10% of LSOAs)": "health_decile",
    "Crime Rank (where 1 is most deprived)": "crime_rank",
    "Crime Decile (where 1 is most deprived 10% of LSOAs)": "crime_decile",
    "Barriers to Housing and Services Rank (where 1 is most deprived)": "housing_rank",
    "Barriers to Housing and Services Decile (where 1 is most deprived 10% of LSOAs)": "housing_decile",
    "Living Environment Rank (where 1 is most deprived)": "environment_rank",
    "Living Environment Decile (where 1 is most deprived 10% of LSOAs)": "environment_decile"
}
imd = imd.rename(columns=imd_col_map)
imd

In [ ]:
# Join the a-spatial IMD data to the LSOAs (these already have the gmgi data)
gm_gmgi_imd_lsoa = pd.merge(left=gm_gmgi_lsoa, right=imd, left_on="LSOA11CD", right_on="lsoa_2011_code")
# Map the gentrification index (sanity check)
#gm_gmgi_imd_lsoa.explore()
gm_gmgi_imd_lsoa.plot(column="imd_rank", legend=True)

In [ ]:
type(gm_gmgi_imd_lsoa.loc[:,imd_col_map.values()])

Now we have LSOAs with IMD measures. Next join the image embeddings (points, called `points_labels_gdf`) to these LSOAs to get the imd measures. We add to the existing points dataset which was created when we did the gentrification stuff

In [ ]:
# Note we only take the new IMD-related columns from the LSOA geodataframe (hence the .loc) 
# but also need the geometry otherwise gpd can't do the spatial join
points_labels_gdf = gpd.sjoin(
    left_df = points_labels_gdf, 
    right_df = gm_gmgi_imd_lsoa.loc[:, list(imd_col_map.values()) + ["geometry"]],
    how='inner', predicate='within')

In [ ]:
label_col = 'imd_rank'

In [ ]:
ax = gm_gmgi_imd_lsoa.plot(column=label_col)
# Add the points
points_labels_gdf.plot(ax=ax, column=label_col, markersize=10, edgecolor="black", linewidth=0.5)

## Run the IMD model

_Mostly this copies code from the gentrification model. Would be nicer to add it to functions but whatever..._

### Grid search for optimal model and hyper-pareters

Start by applying a few ML models and choosing the best one (and the best parameters). Then later we can use this model to explore the relationship between street view embeddings and deprivation

In [ ]:
# ------------------------------------------------------------------
# Build feature matrix X and target vector y
# ------------------------------------------------------------------
X = np.stack(points_labels_gdf["embedding"].values)     # shape (n_points, embed_dim)
y = points_labels_gdf[label_col].values

print("Feature matrix shape:", X.shape, "Target vector shape:", y.shape)

In [ ]:
# TEMPORARY SAMPLE FOR TESTING
#rng = np.random.default_rng(seed=42)
#idx = rng.choice(len(y), size=100, replace=False)
#X = X[idx]
#y = y[idx]

In [ ]:
if RUN_IMD_MODEL:
    # Split data into training and test sets
    X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
        X , y, np.arange(X.shape[0]), test_size=0.2, random_state=42)
    print(f"Training points: {X_train.shape[0]}, Test points: {X_test.shape[0]}")
    
    # Define model pipelines and parameter grids for cross-validation
    models = []
    param_grids = []
    
    # 1. Linear Regression (with standard scaling)
    pipe_linear = Pipeline([
        ('scaler', StandardScaler()),
        ('reg', LinearRegression())
    ])
    # No hyperparameters to tune for plain LinearRegression (we could consider Ridge/Lasso alphas, but skip for simplicity)
    models.append(pipe_linear)
    param_grids.append({})  # empty grid means just evaluate the baseline linear model
    
    # 2. Random Forest Regressor
    pipe_rf = Pipeline([
        ('scaler', StandardScaler()),  # scaler doesn't affect RF but included for uniformity
        ('reg', RandomForestRegressor(random_state=42))
    ])
    
    models.append(pipe_rf)
    param_grids.append({
        'reg__n_estimators': [100, 200],   # try 100 and 200 trees
        'reg__max_depth': [None, 10, 20]  # try unlimited depth and a couple of depth limits
    })
    
    # 3. Neural Network (MLPRegressor)
    pipe_mlp = Pipeline([
        ('scaler', StandardScaler()),
        ('reg', MLPRegressor(max_iter=500, random_state=42))
    ])
    models.append(pipe_mlp)
    param_grids.append({
        'reg__hidden_layer_sizes': [(100,), (100,50)],  # one hidden layer vs two layers
        'reg__alpha': [1e-4, 1e-3]  # L2 regularization strengths
        # (Other hyperparameters like learning_rate_init can be added if needed)
    })
    
    # Perform cross-validation for each model to find the best hyperparameters
    best_model = None
    best_score = -np.inf
    best_model_name = None
    best_params = {}
    
    print("Training models")
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    ncores = min(multiprocessing.cpu_count()-1, 100)
    print(f"USing {ncores} cores.")  # Take all cores but one, and not more than 100 (don't want to kill the HPC)
    for model, param_grid, name in zip(models, param_grids, ["LinearReg", "RandomForest", "NeuralNet"]):
        print(f"\tTraining: {model}...")
        if param_grid:
            # Use GridSearchCV for models with hyperparameters
            grid = GridSearchCV(model, param_grid, cv=cv, scoring='r2', n_jobs=ncores)
            grid.fit(X_train, y_train)
            cv_score = grid.best_score_
            model_best = grid.best_estimator_
            params_best = grid.best_params_
        else:
            # For Linear Regression (no params to tune), just do cross_val_score
            scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
            cv_score = np.mean(scores)
            model.fit(X_train, y_train)  # train on full training data
            model_best = model
            params_best = {}
        print(f"{name} CV mean R^2 = {cv_score:.3f} {('(best params: '+str(params_best)+')') if params_best else ''}")
        # Track the best model
        if cv_score > best_score:
            best_score = cv_score
            best_model = model_best
            best_model_name = name
            best_params = params_best.copy()         # <- capture the *winning* params

    
    print(f"Best model from CV: {best_model_name} with R^2 = {best_score:.3f} and params {best_params}")

    # Save the meta-data so the model can be trained on the different domains of deprivation
    best_model_info = {
        "model_name": best_model_name,
        "best_score": float(best_score),
        "best_params": best_params
    }
    with open(os.path.join(data_dir, "5-imd_best_model_info.json"), "w") as f:
        json.dump(best_model_info, f, indent=2)
    print(f"Cached model info:", best_model_info)

    ## ---- SAVE the model ----
    #bundle = {
    #    "model": best_model,                 # the fitted Pipeline
    #    "name": best_model_name,             # e.g., "RandomForest"
    #    "cv_score_r2": float(best_score),    # ensure JSON-safe
    #    "trained_at": datetime.utcnow().isoformat() + "Z",
    #    "sklearn_version": sklearn.__version__
    #}
    #dump(bundle, os.path.join(data_dir, "5-imd_best_model_bundle.joblib", compress=3)
    #print("Saved to best_model_bundle.joblib")
    
else:
    print("RUN_IMD_MODEL is False so not running the gentrification predictor model")

In [ ]:
## ---- LOAD the model ----
#if RUN_IMD_MODEL:
#    bundle = load("best_model_bundle.joblib")
#    print("Loading cached model")
#    best_model = bundle["model"]
#
#    print("\t", bundle["name"], bundle["cv_score_r2"], bundle["trained_at"], bundle["sklearn_version"])

Run the best model on the test set to get a reliable error estimate

In [ ]:
if RUN_IMD_MODEL:
    # Ensure best_model is trained on the entire training set (GridSearchCV already did refit; for linear we did manually)
    # If best_model was from cross_val_score (Linear), we already called model.fit above.
    
    # Predict on test set
    y_pred = best_model.predict(X_test)
    
    # Evaluate performance
    r2_test = r2_score(y_test, y_pred)
    # rmse_test = mean_squared_error(y_test, y_pred, squared=False)  # (not avialable in older skikitlearn)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"\nTest R^2 score: {r2_test:.3f}")
    print(f"Test RMSE: {rmse_test:.3f}")
    
    # Attach predictions to the test points for mapping
    test_points = points_labels_gdf.iloc[test_idx].copy()
    test_points['predicted'] = y_pred
    test_points['error'] = test_points['predicted'] - test_points[label_col]

In [ ]:
if RUN_IMD_MODEL:
    # Plot the actual gentrification by area
    fig, ax = plt.subplots(figsize=(8, 6))
    gm_gmgi_imd_lsoa.plot(column=label_col, ax=ax, legend=True, cmap='plasma', edgecolor='gray')
    ax.set_title("Actual IMD by Area")
    ax.axis('off')
    plt.show()
    
    # Plot the prediction errors at test points
    fig, ax = plt.subplots(figsize=(8, 6))
    # Plot polygons outlines for context
    gm_gmgi_imd_lsoa.boundary.plot(ax=ax, color='lightgray')
    # Plot test points with error coloration
    test_points.plot(column='error', ax=ax, legend=True, cmap='coolwarm', markersize=50)
    ax.set_title("Model Prediction Error at Test Points")
    ax.axis('off')
    plt.show()

Now we have the best model type and optimal hyper-parameters. Now use that modal type and hyper-parameters to predict the different domains of deprivation. We cached the model and parmaeters so don't need to re-run the GridSearchCV.

In [ ]:
assert os.path.exists(os.path.join(data_dir, "5-imd_best_model_info.json")), \
    "IMD grid search params json file doesn't exist, need to run the grid search - set RUN_IMD_MODEL to True and re-run)"

with open(os.path.join(data_dir, "5-imd_best_model_info.json")) as f:
    info = json.load(f)
    print("Loaded model info:", info)

# Recreate a fresh model with the same hyperparameters
model_map = {
    "LinearReg": LinearRegression,
    "RandomForest": RandomForestRegressor,
    "NeuralNet": MLPRegressor
}

ModelClass = model_map[info["model_name"]]

params = {k.replace("reg__", ""): v for k, v in info["best_params"].items()} # Strip any 'reg__' prefixes (used by Pipeline)
model = ModelClass(**params)



### Running IMD models on different domains of deprivation

Nowe we have a model and optimal hyper-parameters, estimate new models that can predict both the overall deprivation as well as the individual domains

[ ] TODO Run model using different predictor

In [ ]:
# Now re-train on different domains of deprivation
#XXXX HERE

### Aggregating the image imbeddings

Currently we have large numbers of images (and embeddings) per LSOA. But each LSOA only has a single deprivation value

[ ] TODO Try aggregating the image embeddings to LSOA and then predicting the model